In [1]:
import netCDF4 as nc

import matplotlib.pyplot as plt
import numpy as np
import os
import re

%matplotlib inline

resultsDir='/data/eolson/MEOPAR/SS2DSOGruns/'
N2chl=1.600

In [2]:
TS=nc.Dataset(os.path.join(resultsDir,'TS_uniform.nc'))

In [3]:
# read z, T, S into CTD
CTD=np.loadtxt('/data/eolson/SOG/SOG-initial/ctd/SG-S3-2004-10-19.sog',skiprows=12,usecols=(1, 4, 2, 8))
# add z=0 at first row, repeating T,S values from next level
data=np.vstack((CTD[0,:],CTD))
data[0,0]=0.
# interpolate to T-points
data_f=(data[0:401:4,:]+data[1:402:4,:])/2

In [4]:
new_ts=nc.Dataset('/data/eolson/MEOPAR/SS2DSOGruns/TS_400m_constS.nc','w')
new_ts.createDimension('y', 10)
new_ts.createDimension('x', 1100)
new_ts.createDimension('deptht', 101)
new_ts.createDimension('time_counter', None)

<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time_counter', size = 0

In [5]:
new_tc=new_ts.createVariable('time_counter',float,('time_counter'),zlib=True)
new_tc.setncattr('units',TS.variables['time_counter'].units)
new_tc.setncattr('long_name',TS.variables['time_counter'].long_name)
new_tc[:]=TS.variables['time_counter']

print (TS.variables['time_counter'])
print( new_ts.variables['time_counter'])

<class 'netCDF4._netCDF4.Variable'>
float32 time_counter(time_counter)
    units: seconds since 2003-08-09 0:00:00
    long_name: Time axis
unlimited dimensions: time_counter
current shape = (1,)
filling on, default _FillValue of 9.969209968386869e+36 used

<class 'netCDF4._netCDF4.Variable'>
float64 time_counter(time_counter)
    units: seconds since 2003-08-09 0:00:00
    long_name: Time axis
unlimited dimensions: time_counter
current shape = (1,)
filling on, default _FillValue of 9.969209968386869e+36 used



In [6]:
new_z=new_ts.createVariable('deptht',float,('deptht'),zlib=True)
new_z.setncattr('units',TS.variables['deptht'].units)
new_z.setncattr('long_name',TS.variables['deptht'].long_name)
new_z.setncattr('positive',TS.variables['deptht'].positive)
zs=np.arange(0.5,404,4)
new_z[:]=zs

print( TS.variables['deptht'])
print (new_ts.variables['deptht'])

<class 'netCDF4._netCDF4.Variable'>
float32 deptht(deptht)
    long_name: Depth
    units: m
    positive: down
unlimited dimensions: 
current shape = (40,)
filling on, default _FillValue of 9.969209968386869e+36 used

<class 'netCDF4._netCDF4.Variable'>
float64 deptht(deptht)
    units: m
    long_name: Depth
    positive: down
unlimited dimensions: 
current shape = (101,)
filling on, default _FillValue of 9.969209968386869e+36 used



In [7]:
new_x=new_ts.createVariable('x',float,('y','x'),zlib=True)
new_x.setncattr('units',TS.variables['x'].units)
new_x.setncattr('long_name',TS.variables['x'].long_name)
new_x=TS.variables['x']

print (TS.variables['x'])
print (new_ts.variables['x'])

<class 'netCDF4._netCDF4.Variable'>
float32 x(y, x)
    long_name: streamwise
    units: metres
unlimited dimensions: 
current shape = (10, 1100)
filling on, default _FillValue of 9.969209968386869e+36 used

<class 'netCDF4._netCDF4.Variable'>
float64 x(y, x)
    units: metres
    long_name: streamwise
unlimited dimensions: 
current shape = (10, 1100)
filling on, default _FillValue of 9.969209968386869e+36 used



In [8]:
new_y=new_ts.createVariable('y',float,('y','x'),zlib=True)
new_y.setncattr('units',TS.variables['y'].units)
new_y.setncattr('long_name',TS.variables['y'].long_name)
new_y=TS.variables['y']

print( TS.variables['y'])
print( new_ts.variables['y'])

<class 'netCDF4._netCDF4.Variable'>
float32 y(y, x)
    long_name: spanwise
    units: metres
unlimited dimensions: 
current shape = (10, 1100)
filling on, default _FillValue of 9.969209968386869e+36 used

<class 'netCDF4._netCDF4.Variable'>
float64 y(y, x)
    units: metres
    long_name: spanwise
unlimited dimensions: 
current shape = (10, 1100)
filling on, default _FillValue of 9.969209968386869e+36 used



In [9]:
# Remove records with negative data values (typically -99.0 or
# -99999) because that indicates invalid data
data_qty=data_f[:,2]
mask = (data_qty >= 0.0)
data_records = len(data_qty[mask])
qty_clean = data_qty[mask]
depth_clean= data_f[:,0][mask]
# Calculate depth and quantity differences from field data for use
# in interpolation
del_depth = depth_clean[1:data_records] - depth_clean[0:data_records-1]
del_qty = qty_clean[1:data_records] - qty_clean[0:data_records-1]
# Interpolate quantity values at grid point depths
i_data = 1
T=0.0*zs
for i_g in range(0, len(zs)):
    if (zs[i_g] > depth_clean[i_data]):
        while ((zs[i_g] > depth_clean[i_data]) and (i_data<data_records-1)):
            i_data = i_data + 1
    T[i_g] = qty_clean[i_data-1] + del_qty[i_data-1] \
        * ((zs[i_g] - depth_clean[i_data-1]) \
        / del_depth[i_data-1])
print (T)

[ 12.0611   12.5737   12.58305  11.92535  11.2438   10.9357   10.59925
  10.5561   10.52095  10.3964   10.29295  10.20695  10.1203   10.09005
  10.07945  10.1169   10.19495  10.2058   10.2228   10.2671   10.27015
  10.2894   10.3184   10.33585  10.34575  10.3721   10.35385  10.34285
  10.31205  10.25455  10.23735  10.20875  10.19735  10.136    10.12665
  10.10935  10.08135  10.04315  10.03595  10.0208   10.00385   9.99255
   9.9715    9.95565   9.94995   9.94845   9.9429    9.93205   9.9218
   9.9131    9.90725   9.90525   9.9011    9.90085   9.89475   9.8952
   9.89345   9.89455   9.89135   9.8949    9.89845   9.902     9.90555
   9.9091    9.91265   9.9162    9.91975   9.9233    9.92685   9.9304
   9.93395   9.9375    9.94105   9.9446    9.94815   9.9517    9.95525
   9.9588    9.96235   9.9659    9.96945   9.973     9.97655   9.9801
   9.98365   9.9872    9.99075   9.9943    9.99785  10.0014   10.00495
  10.0085   10.01205  10.0156   10.01915  10.0227   10.02625  10.0298
  10.03335 

In [10]:
# Remove records with negative data values (typically -99.0 or
# -99999) because that indicates invalid data
data_qty=data_f[:,3]
mask = (data_qty >= 0.0)
data_records = len(data_qty[mask])
qty_clean = data_qty[mask]
depth_clean= data_f[:,0][mask]
# Calculate depth and quantity differences from field data for use
# in interpolation
del_depth = depth_clean[1:data_records] - depth_clean[0:data_records-1]
del_qty = qty_clean[1:data_records] - qty_clean[0:data_records-1]
# Interpolate quantity values at grid point depths
i_data = 1
S=0.0*zs+30.0
print (S)

[ 30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.
  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.
  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.
  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.
  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.
  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.
  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.]


In [11]:
vecT=np.reshape(T,(101,1,1))
data_T=np.tile(vecT,(1,1,10,1100))

new_T=new_ts.createVariable('votemper',float,('time_counter','deptht','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
new_T.setncattr('long_name','T')
new_T.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_T[:,:,:,:]=data_T
print (TS.variables['vosaline'])
print (new_ts.variables['votemper'])

<class 'netCDF4._netCDF4.Variable'>
float32 vosaline(time_counter, deptht, y, x)
    units: none
    long_name: Practical Salinity
    coordinates: x y deptht time_counter
    grid: 2D bathy
unlimited dimensions: time_counter
current shape = (1, 40, 10, 1100)
filling on, default _FillValue of 9.969209968386869e+36 used

<class 'netCDF4._netCDF4.Variable'>
float64 votemper(time_counter, deptht, y, x)
    long_name: T
    coordinates: x y deptht time_counter
unlimited dimensions: time_counter
current shape = (1, 101, 10, 1100)
filling on, default _FillValue of 9.969209968386869e+36 used



In [12]:
vecS=np.reshape(S,(101,1,1))
data_S=np.tile(vecS,(1,1,10,1100))

new_S=new_ts.createVariable('vosaline',float,('time_counter','deptht','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
new_S.setncattr('long_name','S')
new_S.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_S[:,:,:,:]=data_S
print (TS.variables['vosaline'])
print (new_ts.variables['vosaline'])

<class 'netCDF4._netCDF4.Variable'>
float32 vosaline(time_counter, deptht, y, x)
    units: none
    long_name: Practical Salinity
    coordinates: x y deptht time_counter
    grid: 2D bathy
unlimited dimensions: time_counter
current shape = (1, 40, 10, 1100)
filling on, default _FillValue of 9.969209968386869e+36 used

<class 'netCDF4._netCDF4.Variable'>
float64 vosaline(time_counter, deptht, y, x)
    long_name: S
    coordinates: x y deptht time_counter
unlimited dimensions: time_counter
current shape = (1, 101, 10, 1100)
filling on, default _FillValue of 9.969209968386869e+36 used



In [13]:
TS.close()
new_ts.close()